In [ ]:
import torch
import torch.nn as nn

from encoders import CLIPResNetEncoder, CLIPImageEncoder

In [ ]:
test_model = CLIPResNetEncoder()

batch_size = 32
dummy_input = torch.randn(batch_size, 3, 224, 224)

output = test_model(dummy_input)
output.size()

In [ ]:
test_model = CLIPImageEncoder()

batch_size = 32
dummy_input = torch.randn(batch_size, 3, 224, 224)

output = test_model(dummy_input)
output.size()